In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import Sequential

from sklearn.experimental import enable_iterative_imputer   # for enabling the use of iterative imputer ( experimental module )
from sklearn.impute import IterativeImputer

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [59]:
filepath = ".../titanic/"
real_vaues = pd.read_csv(filepath + "results.csv", index_col='PassengerId')

train_data = pd.read_csv(filepath + "train.csv", index_col='PassengerId') #, index_col='PassengerId'
train_data['Sex']=train_data['Sex']=='female'

test_data = pd.read_csv(filepath + "test.csv", index_col='PassengerId') #, index_col='PassengerId'
test_data['Sex']=test_data['Sex']=='female'

FEATURES=['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Cabin_letter', 'Title']

tt_data = train_data.drop(columns=['Survived']).append(test_data)

print('done')

done


In [79]:
enc = OneHotEncoder(handle_unknown='ignore')
import re

tt_data['Title'] = tt_data['Name'].astype(str).map(lambda x: re.split(', |\. ', x)[1])
tt_data.Title[~tt_data.Title.isin(['Mr','Miss','Mrs','Master'])]='other'

tt_data['Cabin_letter'] = tt_data['Cabin'].astype(str).str[0]

tt_data['Ticket_Frequency'] = tt_data.groupby('Ticket')['Ticket'].transform('count')
tt_data['Ticket_Frequency'][tt_data['Ticket_Frequency']>=5] = 5

tt_data['Family_size'] = tt_data['SibSp'] + tt_data['Parch']
tt_data['Family_size'] = tt_data['Family_size'].map(lambda x: 1 if x==0 else (2 if x<5 else 3))

tt_encoded = enc.fit_transform(tt_data[['Title', 'Cabin_letter', 'Embarked', 'Pclass']]).toarray()
df_tt_encoded = pd.DataFrame(data = tt_encoded, index = tt_data.index)

print(df_tt_encoded.shape, df_tt_encoded.shape)
print(train_data.shape, test_data.shape)

(1309, 21) (1309, 21)
(891, 11) (418, 10)


C:\Users\movse\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\movse\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [88]:
tt_full = df_tt_encoded.join(tt_data[['Sex', 'Age', 'Fare']]) #, 'Ticket_Frequency', 'Family_size'
train_full = tt_full.loc[train_data.index]
test_full = tt_full.loc[test_data.index]


print(train_full.shape, test_full.shape, tt_full.shape)

(891, 24) (418, 24) (1309, 24)


In [89]:
it_imputer = IterativeImputer(max_iter=10, initial_strategy = 'median')
tt_iter = pd.DataFrame(data = it_imputer.fit_transform(tt_full), columns = tt_full.columns, index = tt_full.index)


train_iter = tt_iter.loc[train_data.index]
test_iter = tt_iter.loc[test_data.index]
#print(train_iter.isnull().sum(), test_iter.isnull().sum(), tt_full.isnull().sum())

In [ ]:
end_model = RandomForestClassifier(criterion='gini',
                                   n_estimators=1750,
                                   max_depth=7,
                                   min_samples_split=6,
                                   min_samples_leaf=6,
                                   max_features='auto',
                                   #bootstrap=True, same result
                                   oob_score=True,
                                   random_state=0,
                                   n_jobs=-1,
                                   verbose=1)

end_model.fit(train_iter, train_data['Survived'])

ans_val = (end_model.predict(test_iter)>0.5) * 1
print('done')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1750 out of 1750 | elapsed:    1.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1226 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 1750 out of 1750 | elapsed:    0.0s finished


done


In [ ]:
answer_pd=pd.DataFrame({'Survived':ans_val}, index = test_data.index)

#answer_pd.to_csv(filepath + 'solution_V5_6.csv')
print('ans = ', (answer_pd==real_vaues)['Survived'].sum()*100/test_data.shape[0], '%')

ans =  78.4688995215311 %


In [98]:
scaler = StandardScaler()
features_nn = [ 'Age',  'Fare']
scaler.fit(tt_iter[features_nn])
df=pd.DataFrame(data = scaler.transform(train_iter[features_nn]), columns = features_nn, index = train_iter.index)
df1 = (train_iter.drop(columns = features_nn)-0.1).join(df)
df3=pd.DataFrame(data = scaler.transform(test_iter[features_nn]), columns = features_nn, index = test_iter.index)
df4 = (test_iter.drop(columns = features_nn)-0.1).join(df3)


end_nn_model = Sequential()
end_nn_model.add(Dense(10, input_shape = [df1.shape[1]], bias_initializer=tf.random.normal ))    # you could write imput_dim = ..., <-(in new versions) activation = 'relu' etc.
#end_nn_model.add(Dense(15, bias_initializer=tf.random.normal))
end_nn_model.add(Dense(1)) #output

end_nn_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

end_nn_model.fit(df1, train_data['Survived'], verbose=1, epochs = 120)

Train on 891 samples
Epoch 1/120
891/891 [==============================] - 0s 237us/sample - loss: 0.8133
Epoch 2/120
891/891 [==============================] - 0s 41us/sample - loss: 0.7110
Epoch 3/120
891/891 [==============================] - 0s 41us/sample - loss: 0.6391
Epoch 4/120
891/891 [==============================] - 0s 41us/sample - loss: 0.5873
Epoch 5/120
891/891 [==============================] - 0s 41us/sample - loss: 0.5467
Epoch 6/120
891/891 [==============================] - 0s 40us/sample - loss: 0.5169
Epoch 7/120
891/891 [==============================] - 0s 41us/sample - loss: 0.4956
Epoch 8/120
891/891 [==============================] - 0s 40us/sample - loss: 0.4796
Epoch 9/120
891/891 [==============================] - 0s 41us/sample - loss: 0.4682
Epoch 10/120
891/891 [==============================] - 0s 42us/sample - loss: 0.4596
Epoch 11/120
891/891 [==============================] - 0s 40us/sample - loss: 0.4533
Epoch 12/120
891/891 [===================

In [99]:
answers = end_nn_model.predict(df4)>0.5
answers = answers*1   # true false to 1 and 0 
answers = answers.reshape(answers.shape[0])
answer_pd = pd.DataFrame( {'Survived' : answers}, index = test_data.index)
answer_pd.to_csv(filepath + 'solution_V6_nn_1.csv')
print('ans = ', (answer_pd==real_vaues)['Survived'].sum()*100/test_data.shape[0], '%')

ans =  80.14354066985646 %


In [100]:
answers = end_nn_model.predict(df1)>0.5
answers = answers*1   # true false to 1 and 0 
answers = answers.reshape(answers.shape[0])
answer_pd = pd.DataFrame( {'Survived' : answers}, index = train_data.index)
#answer_pd.to_csv(filepath + 'solution_V6_nn_OHE.csv')
print('ans = ', (answer_pd==pd.DataFrame(train_data.Survived))['Survived'].sum()*100/train_data.shape[0], '%')

ans =  83.05274971941638 %


In [101]:


tt_data['Family_size'] = tt_data['SibSp'] + tt_data['Parch']
tt_data['Family_size'] = tt_data['Family_size'].map(lambda x: 1 if x==0 else (2 if x<5 else 3))

In [ ]:
tt_data.Family_size.value_counts()

1    790
2    459
3     60
Name: Family_size, dtype: int64